In [2]:
import cv2
import numpy as np
from yolov4.tf import YOLOv4
import time

Call tf.config.experimental.set_memory_growth(GPU0, True)


## Источники

Скачать веса разных версий yolov3 - - https://pjreddie.com/media/files/yolov3.weights

Скачать архитектуру и веса yolv4  - веса https://github.com/AlexeyAB/darknet (официальный репозиторий yolo)

Вся модель, написанная на tensroflow - http://datahacker.rs/tensorflow2-0-yolov3/

In [3]:
class_names =  ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
    "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
    "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
    "banana","apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake","chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", 
    "mouse","remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator","book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

## OpenCV inference (cpu)

Источник - https://gist.github.com/YashasSamaga/e2b19a6807a13046e399f4bc3cca3a49

In [ ]:
#cap.release()
cv2.destroyAllWindows()

In [4]:
white_list=['person']

CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.4
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]


vc = cv2.VideoCapture("humans_1.mp4")

net = cv2.dnn.readNet("yolov4-tiny.weights", "yolov4-tiny.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

while cv2.waitKey(1) < 1:
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()

    start = time.time()
    classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    end = time.time()

    start_drawing = time.time()
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid[0]], score)
        if class_names[classid[0]] in white_list:
            color=(255,0,0)
            cv2.rectangle(frame, box, color, 2)
            cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()
    
    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)
    #time.sleep(0.1)

KeyboardInterrupt: 

## Tensorflow inference (gpu)

Источник - https://wiki.loliot.net/docs/lang/python/libraries/yolov4/python-yolov4-about/

In [4]:
yolo = YOLOv4()

yolo.config.parse_names("coco.names")
yolo.config.parse_cfg("yolov4-tiny-tf.cfg")

yolo.make_model()
yolo.load_weights("yolov4-tiny.weights", weights_type="yolo")
#yolo.summary(summary_type="yolo")
#yolo.summary()

In [ ]:
#help(YOLOv4)

### Автоматический инференс

In [ ]:
#yolo.inference(media_path="me3.jpg", is_image=True,cv_frame_size=(640, 480))
yolo.inference(media_path="humans_1.mp4", is_image=False)


### Ручной инференс

In [ ]:
#cap.release()
cv2.destroyAllWindows()

In [ ]:
white_list=['person']
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

vc = cv2.VideoCapture("humans_1.mp4")
while cv2.waitKey(1) < 1:
    
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()


    start = time.time()
    results = yolo.predict(frame, 0.2)
    end = time.time()

    start_drawing = time.time()
    for res in results:
        
        x,y,w,h,classid,score=res
        x=x*frame.shape[1]
        y=y*frame.shape[0]
        w=w*frame.shape[1]
        h=h*frame.shape[0]

        x1y1=(int(x-w/2),int(y-h/2))
        x2y2=(int(x+w/2),int(y+h/2))

        classid=int(classid)
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid], score)
        if class_names[classid] in white_list:
            color=(255,0,0)
            cv2.rectangle(frame, x1y1,x2y2, color, 2)
            cv2.putText(frame, label, (x1y1[0],x1y1[1]  - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()
    
    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)
    #time.sleep(0.1)

In [ ]:
white_list=['person']
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

vc = cv2.VideoCapture("humans_1.mp4")
while cv2.waitKey(1) < 1:
    
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()


    start = time.time()
    results = yolo.predict(frame, 0.2)
    end = time.time()

    start_drawing = time.time()
    for res in results:
        
        x,y,w,h,classid,score=res
        x=x*frame.shape[1]
        y=y*frame.shape[0]
        w=w*frame.shape[1]
        h=h*frame.shape[0]

        x1y1=(int(x-w/2),int(y-h/2))
        x2y2=(int(x+w/2),int(y+h/2))

        classid=int(classid)
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid], score)
        if class_names[classid] in white_list:
            color=(255,0,0)
            cv2.rectangle(frame, x1y1,x2y2, color, 2)
            cv2.putText(frame, label, (x1y1[0],x1y1[1]  - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()
    
    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)
    #time.sleep(0.1)

### Автоматический инференс

In [7]:
#yolo.inference(media_path="me3.jpg", is_image=True,cv_frame_size=(640, 480))
yolo.inference(media_path="humans_1.mp4", is_image=False)


KeyboardInterrupt: 

### Ручной инференс

In [8]:
#cap.release()
cv2.destroyAllWindows()

In [ ]:
white_list=['person']
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

vc = cv2.VideoCapture("humans_1.mp4")
while cv2.waitKey(1) < 1:
    
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()


    start = time.time()
    results = yolo.predict(frame, 0.2)
    end = time.time()

    start_drawing = time.time()
    for res in results:
        
        x,y,w,h,classid,score=res
        x=x*frame.shape[1]
        y=y*frame.shape[0]
        w=w*frame.shape[1]
        h=h*frame.shape[0]

        x1y1=(int(x-w/2),int(y-h/2))
        x2y2=(int(x+w/2),int(y+h/2))

        classid=int(classid)
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid], score)
        if class_names[classid] in white_list:
            color=(255,0,0)
            cv2.rectangle(frame, x1y1,x2y2, color, 2)
            cv2.putText(frame, label, (x1y1[0],x1y1[1]  - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()
    
    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)
    #time.sleep(0.1)